# An advanced look at LLM Agents
## Notebook 2: Building an agent using the GPT API chat completion endpoint
---

Previously, we relied on LangChain to to build the agent. However, this is not necessary, since an agent is nothing more than a fancy while loop. 

**The goal** 

With this notebook you will see what an agent is under the hood, and you can build it based on the LLM output

🌟 So ... let us begin!  

**Contents:**

1. [Exercise 1: New tool format](#1)
2. [Exercise 2: String vs function call response](#2)
3. [Exercise 3: Understanding the agent while-loop](#3)

In [1]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('.'))
folder_b_path = os.path.join(current_dir, 'helper_functions')
sys.path.append(current_dir)

from helper_functions.tools import my_own_wiki_tool, weather_tool
from helper_functions.keys import client
import json

---

## The tools

When querying OpenAI's API, tools are called via **function calling** (see the API documentation on function calling [here](https://platform.openai.com/docs/assistants/tools/function-calling/quickstart)). Functions have to be passed in JSON format, which we will explore below.


#### Exercise 1: New tool format <a id='1'></a>

First, we'll look at an example of how to format the `my_own_wiki_tool` into a function json, then you'll format the `weather_tool` into a function json the same way.

**TASK:**  
Compile the next two cells and have a look at the example. Do you understand the output?

In [2]:
# Example: print attributes of my_own_wiki_tool

print('name: ', my_own_wiki_tool.name)
print('description: ', my_own_wiki_tool.description)
print('arguments: ', my_own_wiki_tool.args)

name:  wikipedia
description:  A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
arguments:  {'query': {'title': 'Query', 'description': 'Input search query', 'type': 'string'}}


In [3]:
# Example: json format of the my_own_wiki_tool

my_own_wiki_tool_json = {
    "type": "function",
    "function": {
        "name": my_own_wiki_tool.name,
        "description": my_own_wiki_tool.description,
        "parameters": {
            "type": "object",
            # below all individual function parameters need to be listed
            "properties": {
                'query':{
                    'description':'Input search query.',
                    'type':'string',
                }
            },
            "required": ['query']
        },
    },
}


**TASK:**  
Using the example above for guidance, create the json format for the weather tool. If you have trouble filling in the attributes, first print them from the tool.

In [4]:
weather_tool_json = {
    "type": "function",
    "function": {
        "name": weather_tool.name,
        "description": weather_tool.description,
        "parameters": {
            "type": "object",
            # below all individual function parameters need to be listed
            "properties": {
                'city':{
                    'description':'city name',
                    'type':'string',
                }
            },
            "required": ['query']
        },
    },
}

Next, the jsonized functions are combined in a list, while the actual tools are stored in a dictionary. After this we have prepared the callable functions and are ready to interact with the chat completions endpoint.

**TASK:**  
Add the `weather_tool` by completing the code below

In [5]:
# callable functions 
callable_functions = [my_own_wiki_tool_json, weather_tool_json]

# Store executable functions with their name in dictionary
available_functions = {
    my_own_wiki_tool.name :my_own_wiki_tool, 
    weather_tool.name: weather_tool
    }

---

## The LLM response to input questions

The LLM can respond with two types of answers to input queries:
* a **string** that answers the question, 
* a **function-call** object, which contains information on which function to call with which arguments.

The second one can be used to execute function calls.

#### Exercise 2: String vs function call response <a id='2'></a>

**TASK:**
Compile both questions and compare the answers. Do you understand the difference?


In [6]:
system_prompt = "You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner."

questions = ["what is the meaning of life?","What temperature is it in Paris?"]

for question in questions:
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

  
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    tools = callable_functions,
    messages=messages,
    )

  print(f"Question: {question}")
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")

Question: what is the meaning of life?
Answer: The meaning of life has been a profound question throughout human history, and different cultures, philosophies, and religions offer various interpretations. Some people find meaning in relationships, personal growth, and experiences, while others may seek purpose through spirituality, contributing to society, or achieving personal goals. Ultimately, it can be a deeply personal journey, and many believe that each person must define their own meaning based on their values and beliefs. What are your thoughts on it?
Function call: None

Question: What temperature is it in Paris?
Answer: None
Function call: [ChatCompletionMessageToolCall(id='call_BlHCmEmzfz0Sxe4eAmwWGUpU', function=Function(arguments='{"city":"Paris"}', name='weather'), type='function')]



**TASK:**  
For the function call object above, extract the **name** and the **arguments** of the function call and print them.

In [7]:
for function in response.choices[0].message.tool_calls:
    print('name: ', function.function.name)
    print('arguments: ', function.function.arguments)

name:  weather
arguments:  {"city":"Paris"}


---

## The Agent - a fancy while loop

While the LLM requests function calls we 
* **extract** the **name and arguments** to be called from the initial LLM response,
* **execute** the **function calls**,
* **store** the **output of the function** in the messages object,
* invoke the LLM again, until no function call are requested.

For more details, you can also check out this [OpenAI function calling guide](https://platform.openai.com/docs/guides/function-calling).

#### Exercise 3: Understanding the agent while-loop <a id='3'></a>

**TASK:**
Complete the code below, then compile the question and investigate the output. Do you understand what you see?

In [8]:
question = "which city is bigger: Paris or Munich?"

messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

response = client.chat.completions.create(
  model="gpt-4o-mini",
  tools = callable_functions,
  messages=messages, 
  tool_choice='required'
  )

print('==== Initial LLM response ====')
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

# while the response requests function calls
while response.choices[0].message.tool_calls:
    
  # store response message with all function calls
  response_message = response.choices[0].message
  messages.append(response_message)

  # execute each tool individually
  for tool_call in response.choices[0].message.tool_calls:
    print('==== Function call ====')

    # function name and arguments
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(f'Calling function "{function_name}" with arguments {function_args}.')

    # execute function call 
    function_response = available_functions[function_name].invoke(function_args)
    print(f'Function call response:\n{function_response}\n')

    # append function response to messages
    messages.append({
        "tool_call_id":tool_call.id, 
        "role": "tool", 
        "name": function_name, 
        "content": function_response
    })
    
  # get a new response from LLM
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    tools = callable_functions,
    messages=messages, 
  )

  print('==== Intermediate LLM response ====')
  print(f"Answer: {response.choices[0].message.content} ")
  print(f"Function call: {response.choices[0].message.tool_calls }\n")

print('==== Final LLM response ====')
print("Question: ", question)
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

==== Initial LLM response ====
Answer: None
Function call: [ChatCompletionMessageToolCall(id='call_dVPM6mqeIJjllHtX4klluppS', function=Function(arguments='{"query": "Paris"}', name='wikipedia'), type='function'), ChatCompletionMessageToolCall(id='call_UOttXx83ejw813xNMBMY1ZsG', function=Function(arguments='{"query": "Munich"}', name='wikipedia'), type='function')]

==== Function call ====
Calling function "wikipedia" with arguments {'query': 'Paris'}.
Function call response:
Page: Paris
Summary: Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an official estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. For its leading role in the arts and sci

🌟 Congratulations - you've finished the workshop